In [1]:
import pandas as pd
from matplotlib import pyplot as plt
from tabulate import tabulate
import numpy as np
import re
import locale
locale.setlocale(locale.LC_ALL, '')
import sys
import subprocess
from scipy.optimize import minimize
from matplotlib import rcParams
from scipy.optimize import basinhopping
rcParams['figure.figsize'] = 9, 9

In [2]:
comps = pd.read_excel('tabulated_data4.xlsm', sheet_name='Compositions')
compNames = list(comps)[1:]
comps

Name McLeod-Mix1 McLeod-Mix2 McLeod-Mix3 McLeod-Mix22  \
0            Group      McLeod      McLeod      McLeod       McLeod   
1           Sample        Mix1        Mix2        Mix3        Mix22   
2               SG    0.705971    0.652476    0.609878     0.698439   
3    Inert Free SG    0.560051    0.558915    0.560063     0.681036   
4              CO2       0.005       0.003      0.0131        0.018   
..             ...         ...         ...         ...          ...   
111            NaN         NaN         NaN         NaN          NaN   
112            NaN         NaN         NaN         NaN          NaN   
113            NaN         NaN         NaN         NaN          NaN   
114            NaN         NaN         NaN         NaN          NaN   
115            NaN         NaN         NaN         NaN          NaN   

    McLeod-Mix24 McLeod-Mix25 Simon&Briggs-10PC Simon&Briggs-20PC API 37-Mix2  \
0         McLeod       McLeod      Simon&Briggs      Simon&Briggs      API 37   
1          Mix24        Mix25              10PC              20PC        Mix2   
2       0.741612      0.59657          0.616048          0.678315    0.789937   
3       0.736766     0.592865           0.55378           0.55378    0.718099   
4         0.0061        0.004                 0                 0      0.0209   
..           ...          ...               ...               ...         ...   
111          NaN          NaN               NaN               NaN         NaN   
112          NaN          NaN               NaN               NaN         NaN   
113          NaN          NaN               NaN               NaN         NaN   
114          NaN          NaN               NaN               NaN         NaN   
115          NaN          NaN               NaN               NaN         NaN   

     ... Custom-A6ST1_2000 Custom-A6ST1_1500 Custom-A6ST1_1000  \
0    ...            Custom            Custom            Custom   
1    ...        A6ST1_2000        A6ST1_1500        A6ST1_1000   
2    ...          1.129789          1.132206          1.164653   
3    ...          0.851054          0.835685          0.867391   
4    ...             0.415             0.432            0.4552   
..   ...               ...               ...               ...   
111  ...               NaN               NaN               NaN   
112  ...               NaN               NaN               NaN   
113  ...               NaN               NaN               NaN   
114  ...               NaN               NaN               NaN   
115  ...               NaN               NaN               NaN   

    Custom-A6ST1_500 Custom-D1_4088 Custom-D1_4089 Custom-D1_4848  \
0             Custom         Custom         Custom         Custom   
1          A6ST1_500        D1_4088        D1_4089        D1_4848   
2             1.2437       1.463583       0.949258       0.880221   
3            0.98819       1.465396       0.942719       0.872138   
4             0.4813        0.01068        0.01118        0.01157   
..               ...            ...            ...            ...   
111              NaN            NaN            NaN            NaN   
112              NaN            NaN            NaN            NaN   
113              NaN            NaN            NaN            NaN   
114              NaN            NaN            NaN            NaN   
115              NaN            NaN            NaN            NaN   

    Custom-D1_4469 Li_Ja_Wu-Sample1 Li_Ja_Wu-Sample2  
0           Custom         Li_Ja_Wu         Li_Ja_Wu  
1          D1_4469          Sample1          Sample2  
2         1.480842         0.572925         0.663684  
3         1.484551         0.571767         0.634273  
4          0.01457                0          0.02853  
..             ...              ...              ...  
111            NaN              NaN              NaN  
112            NaN              NaN              NaN  
113            NaN              NaN              NaN  
114            N

In [3]:
def make_valid_filename(input_string):
    valid_filename = re.sub(r'[^\w.-]', '_', input_string)
    valid_filename = valid_filename.strip()
    valid_filename = re.sub(r'[-_.]{2,}', '_', valid_filename)
    valid_filename = valid_filename.replace('@', '_').replace('#', '_')
    return valid_filename

filename ='HC_BIPS_v11'
phz_template = filename+'.phz' # The .Phz file to use as the base

In [7]:
R, mwAir, degF2R, Zc = 10.731577089016, 28.97, 459.67, 0.3074
class mix:
    def __init__(self, name, group, sample, sg, co2, h2s, n2, gas):
        h2 = 0
        self.comps = [co2, h2s, n2, h2, gas]
        self.compnames = ['CO2', 'H2S', 'N2', 'H2', 'Gas']
        self.name = make_valid_filename(name)
        self.group = group
        self.sample = sample
        self.sg = sg
        mws = np.array([44.01, 34.082, 28.014, 2.016, 0])
        self.sg_hc = (sg - (co2 * mws[0] + h2s * mws[1] + n2 * mws[2] + h2*mws[3]) / mwAir) / (1 - co2 - h2s - n2 - h2)
        mw_gas = self.sg_hc * mwAir
        #self.pc = (coefic_pc[0] * mw_gas + coefic_pc[1])/(coefic_pc[2] * mw_gas + coefic_pc[3])
        #self.tc = (coefic_tc[0] * mw_gas + coefic_tc[1] ) /(coefic_tc[2] * mw_gas + coefic_tc[3] )     
        #self.pc = (coefic_pc[0] * mw_gas + coefic_pc[1])
        #self.tc = (coefic_tc[0] * mw_gas + coefic_tc[1] )
        self.expmt = pd.read_excel('tabulated_data4.xlsm', sheet_name=name)
        
    def write_include(self, base_chr): # Write include files for 4 component model
        mws = np.array([44.01, 34.082, 28.014, 2.016, 0])
        df = self.expmt.copy()
        temps = list(set(df['degf'].to_list()))
        temps.sort()
        
        out_str = 'DEFINE MW '+str(self.sg_hc * mwAir)+'\n'
        
        for temp in temps:
            out_str += 'DEFINE degR '+str(temp+459.67)+'\n\n'
            out_str += 'CHARACTERIZATION "CHR_'+self.name+'_'+str(temp)+'" BASE "'+base_chr+'"\n'
            header = ['COMPONENT', 'MW', 'TC,R', 'PC,PSIA', 'AF', 'VTRAN', 'A', 'B']
            rows = []
            for c in range(len(self.compnames)):
                rows.append([self.compnames[c], '', '', '', '', '', '', ''])
            rows[-1][1] = round(self.sg_hc * mwAir, 4)
            #rows[-1][2] = round(self.tc,2)
            #rows[-1][3] = round(self.pc,2)

            out_str += tabulate(rows, header)
            out_str += '\nEND\n\n'
            
            out_str += 'DEFINE degR_Slope CO2_degR_Slope, DEFINE MW_Slope CO2_MW_Slope, DEFINE Intcpt CO2_Intcpt, DEFINE Inert CO2\n'
            out_str += 'INCLUDE bips_calc.inc\n\n'
            
            out_str += 'DEFINE degR_Slope H2S_degR_Slope, DEFINE MW_Slope H2S_MW_Slope, DEFINE Intcpt H2S_Intcpt, DEFINE Inert H2S\n'
            out_str += 'INCLUDE bips_calc.inc\n\n'
            
            out_str += 'DEFINE degR_Slope N2_degR_Slope, DEFINE MW_Slope N2_MW_Slope, DEFINE Intcpt N2_Intcpt, DEFINE Inert N2\n'
            out_str += 'INCLUDE bips_calc.inc\n\n'
            
            out_str += 'DEFINE degR_Slope H2_degR_Slope, DEFINE MW_Slope H2_MW_Slope, DEFINE Intcpt H2_Intcpt, DEFINE Inert H2\n'
            out_str += 'INCLUDE bips_calc.inc\n\n'
            
            ## Inert:Inert BIP Pairs                                                                                                   
            #slopes = [[-0.000879210909090909, -0.00130965, 0.00118182785714286, -0.00150498, 0.00461018571428571, 0.000235450357142857],                                
            #          [0.00009085, 0.000541365245623941, 0.00135899363636364, 0.00103136527385658, 0.00112685210526316, 0.000149872727272727]]                                    
            #intcpts = [[0.239517381818182, -0.085784, 0.203866711904762, 0.85241, 0.400074571428572, 0.204373053571429],                                   
            #           [0.0880919666666667, -0.247960272896669, 0.172206318181818, 0.601450271597967, 0.822955005263158, 0.220908090909091]]                               
            #evals = [np.max, np.max, np.max, np.max, np.min, np.min]                                                                  
            #pairs = ['H2S with CO2', 'CO2 with N2', 'H2S WITH N2', 'CO2 with H2', 'H2S with H2', 'N2 with H2']                       
            #inert_bips = [evals[i]([slopes[0][i]*temp + intcpts[0][i], slopes[1][i]*temp + intcpts[1][i]]) for i in range(len(pairs))]
            #                                                                                                                      
            #for p, pair in enumerate(pairs):                                                                                          
            #    out_str += 'REPLACE BIP of '+pair+' by '+str(inert_bips[p])+'\n'  
                
            #BIP_HC = max(0.000160492*temp + 0.056787492, 1/(0.040933216*temp + 2.454878276)) 
            #BIP_CN = 1/(-0.061940854*temp + -0.494913095)                     
            #BIP_HN = -0.836144851 + (temp * 2.80825315) / (348.2693873 + temp)

            #out_str += 'REPLACE BIP of H2S with CO2 by '+str(BIP_HC)+'\n'
            #out_str += 'REPLACE BIP of CO2 with N2 by '+str(BIP_CN)+'\n'
            #out_str += 'REPLACE BIP of H2S with N2 by '+str(BIP_HN)+'\n\n'
            
            out_str += 'REPLACE BIP of CO2 with H2S by CO2_H2S\n'
            out_str += 'REPLACE BIP of CO2 with N2 by CO2_N2\n'
            out_str += 'REPLACE BIP of CO2 with H2 by CO2_H2\n'
            out_str += 'REPLACE BIP of H2S with N2 by H2S_N2\n'
            out_str += 'REPLACE BIP of H2S with H2 by H2S_H2\n'
            out_str += 'REPLACE BIP of N2 with H2 by N2_H2\n\n'
                
            out_str += '\nINCLUDE '+self.name+'_init.inc\n\n'
            
            out_str += 'MIX '+self.name+'_'+str(temp)+'\n'
            
            rows = []
            for c in range(len(self.compnames)):
                rows.append([self.compnames[c], self.comps[c]])
            out_str += tabulate(rows)
            out_str += '\nEND\n\n'
        
            out_str += 'QIX FEED 1 MOLE '+self.name+'_'+str(temp)+'\n'
            out_str += 'CCE ID "'+self.name+'_'+str(temp)+'"\n'

            exp_df = pd.DataFrame()
        
            filterd_df = df[df['degf'] == temp].copy()
            temps = filterd_df['degf'].to_list()+['Wt']
            press = filterd_df['psia'].to_list()+['']
            zs = filterd_df['Z'].to_list()+[1]
            gss = [1 for z in zs]
            gss[-1] = 0
            
            exp_df['TEMP (F)']= temps
            exp_df['PRES (psia)']= press
            exp_df['Z']= zs
            exp_df['GS']= gss

            out_str += tabulate(exp_df, headers='keys', showindex='never', numalign='left')
            out_str += '\nEND\n\n'

        return out_str


In [5]:
#phz_template = 'Inerts_PR_Match_VSHIFT_Only_AB_CO2_BigLoop_BIPS_Temp_reduced_MW_Slope3_Orig_Omega.phz' # The .Phz file to use as the base


Phz_loc = "C:\\Temp\\PhazeComp\\phazecomp.exe"         # The location of the PhazeComp executable on your system
#Phz_loc = "C:\\Program Files\\PhazeComp\\1.81\\phazecomp.exe"         # The location of the PhazeComp executable on your system

def create_phz(phz_template):
    rms=float("nan")
    try:
        proc = subprocess.run([Phz_loc, phz_template,'NUL',">"], capture_output=True)
    except:
        print("Either phazecomp.exe couldnt be found (is it in "+Phz_loc+" ?), or "+phz_template+" couldn't be found - Exiting..")
        sys.exit()
    
    found_rms = False
    for i in range(len(proc.stderr.splitlines())-1,0,-1):
        line = proc.stderr.splitlines()[i].decode("utf-8")
        if "% RMS average weighted" in line:
            rms = float(line.split()[2])
            found_rms = True
            break
    if not found_rms:
        print("Error: No RMS for ",phz_template)
    return rms

In [6]:
mw_hcs = [16.043, 27.91, 34.51, 41.63]         

comp_dic = {}                                                                                     
#print(x)                                                                                         
for c in range(len(compNames)):                                                                   
    name = compNames[c]                                                                           
    group = comps.iloc[0, c+1]                                                                    
    sample = comps.iloc[1, c+1]                                                                   
    sg = comps.iloc[2, c+1]                                                                       
    co2 = comps.iloc[4, c+1]                                                                      
    h2s = comps.iloc[5, c+1]                                                                      
    n2 = comps.iloc[6, c+1]                                                                       
    gas = 1 - co2 - h2s - n2                                                                      
    comp_dic[name] = mix(name, group, sample, sg, co2, h2s, n2, gas)        
                                                                                                  
# 4 component include files                                                                       
for c in range(len(compNames)):                                                                   
    file_path = make_valid_filename(compNames[c])                                                 
    string = comp_dic[compNames[c]].write_include('RealGas')                                      
    with open('.\\'+filename+'\\'+file_path+'.inc', 'w') as file:                                 
        print('.\\'+filename+'\\'+file_path+'.inc')                                               
        file.write(string)                                                                        
                                                                                                  
#gergs = ['lean', 'leanmid', 'mid', 'rich']                                                       
#gerg_mws = [16.043, 25.977, 33.864, 45.08779652]                                                 
                                                                                                  
#for g, gerg in enumerate(gergs):                                                                 
#    out_str = 'DEFINE MW '+str(mw_hcs[g])+'\n'                                                   
#    out_str += 'DEFINE mult_tc '+str(1/(coefic_tc[0] * gerg_mws[g] + coefic_tc[1]) )+'\n'        
#    out_str += 'DEFINE mult_pc '+str(1/(coefic_pc[0] * gerg_mws[g] + coefic_pc[1]))+'\n'         
#    with open('.\\GERG\\'+gerg+'_init.inc', 'w') as file:                                        
#        file.write(out_str)                                                                      
                                                                                                  
#rms = create_phz(phz_template)                                                                   
#print(x, rms)                                                                                    
#return rms                                                                                       

.\HC_BIPS_v11\McLeod-Mix1.inc
.\HC_BIPS_v11\McLeod-Mix2.inc
.\HC_BIPS_v11\McLeod-Mix3.inc
.\HC_BIPS_v11\McLeod-Mix22.inc
.\HC_BIPS_v11\McLeod-Mix24.inc
.\HC_BIPS_v11\McLeod-Mix25.inc
.\HC_BIPS_v11\Simon_Briggs-10PC.inc
.\HC_BIPS_v11\Simon_Briggs-20PC.inc
.\HC_BIPS_v11\API_37-Mix2.inc
.\HC_BIPS_v11\API_37-Mix17.inc
.\HC_BIPS_v11\API_37-Mix25.inc
.\HC_BIPS_v11\API_37-Mix26.inc
.\HC_BIPS_v11\API_37-Mix402.inc
.\HC_BIPS_v11\API_37-Mix601.inc
.\HC_BIPS_v11\API_37-Mix705.inc
.\HC_BIPS_v11\API_37-Mix707.inc
.\HC_BIPS_v11\API_37-Mix722.inc
.\HC_BIPS_v11\API_37-Mix727.inc
.\HC_BIPS_v11\API_37-Mix729.inc
.\HC_BIPS_v11\API_37-Mix735.inc
.\HC_BIPS_v11\Satter_Campbell-MixA.inc
.\HC_BIPS_v11\Satter_Campbell-MixB.inc
.\HC_BIPS_v11\Satter_Campbell-MixC.inc
.\HC_BIPS_v11\Satter_Campbell-MixD.inc
.\HC_BIPS_v11\Buxton_Campbell-Mix1.inc
.\HC_BIPS_v11\Buxton_Campbell-Mix2.inc
.\HC_BIPS_v11\Buxton_Campbell-Mix3.inc
.\HC_BIPS_v11\Buxton_Campbell-Mix4.inc
.\HC_BIPS_v11\Buxton_Campbell-Mix5.inc
.\HC_BIPS_v11\R

In [9]:
def norm_x0(x, bounds):
    x0 = []
    for i, bound in enumerate(bounds):
        span = bound[1] - bound[0]
        x0.append((x[i] - bound[0])/span)
    return x0

def denorm_solx(x, bounds):
    x0 = []
    for i, bound in enumerate(bounds):
        span = bound[1] - bound[0]
        x0.append(x[i]*span + bound[0])
    return x0
    

In [13]:
def fn2(x):
    x = denorm_solx(x, bounds)
    v1, v2 = x[:2]
    Offset, Pl = x[2:] 

    mw_ch4, tc_ch4, pc_ch4 = 16.043, 343.008, 667.029        
    Vl = tc_ch4*(mw_ch4 + Offset + Pl)/(mw_ch4 + Offset)

    vc_ch4 = Zc * R * tc_ch4 / pc_ch4
    v3 = vc_ch4 - v1*mw_ch4**2 - v2*mw_ch4 

    # 5 component include files
    for comp in compNames:
        file_path = make_valid_filename(comp)
        mw = comp_dic[comp].sg_hc * mwAir
        
        tc = (mw + Offset)*Vl/(mw + Offset + Pl)
        Vc = v1*mw**2 + v2*mw + v3       
        pc = Zc * R * tc / Vc
            
        out_str = 'REPLACE TC of Gas by '+str(tc)+' R\n'
        out_str += 'REPLACE PC of Gas by '+str(pc)+' psia\n'
        
        with open('.\\'+filename+'\\'+file_path+'_init.inc', 'w') as file:
            #print('.\\'+filename+'\\'+file_path+'_init.inc')
            file.write(out_str)
    
    gergs = ['lean', 'leanmid', 'mid', 'rich']
    gerg_mws = [16.043, 27.91, 34.51, 41.63]
    gerg_tcs = [343.00800000000004, 485.5369688556304, 548.7898063019243, 608.7178981327803]
    gerg_pcs = [674.863955121382, 677.3035189776824, 654.0642908899845, 623.4933424017122]

    for g, gerg in enumerate(gergs):
        mw = gerg_mws[g]
        tc = (mw + Offset)*Vl/(mw + Offset + Pl)
        Vc = v1*mw**2 + v2*mw + v3       
        pc = Zc * R * tc / Vc

        out_str = 'REPLACE TC of Gas by '+str(tc)+' R\n'
        out_str += 'REPLACE PC of Gas by '+str(pc)+' psia\n'
        with open('.\\'+filename+'_GERG\\'+gerg+'_gas_init.inc', 'w') as file:
            file.write(out_str)
        
    rms = create_phz(phz_template)
    print(x, rms)
    return rms

In [18]:
x0 = [-12.590960856075188, -0.018583796693770138, 9.099574152165, -0.00270020030985145]
fn2(norm_x0(x0, bounds))

[-12.590960856075188, -0.018583796693770138, 9.099574152165, -0.00270020030985145] 1.088


1.088

In [21]:
#      v1                      v2           Offset         Pl
x0 = [8.02981653523445e-05, 0.056591187111412006, 11.171973798842838, 183.60608782036712]
bounds = [(5e-5, 1e-4), (0.05, 0.062), (10.5, 11.8), (182, 185)]
x = norm_x0(x0, bounds)

sol = minimize(fn2, x, method='Nelder-Mead', bounds=[(0, 1), (0, 1), (0, 1), (0, 1)])
# 1.184 with v6 + H2 BIP's. Now trying with constant BIP's

[8.02981653523445e-05, 0.056591187111412006, 11.171973798842838, 183.60608782036712] 0.9098
[8.181307361996171e-05, 0.056591187111412006, 11.171973798842838, 183.60608782036712] 0.91
[8.02981653523445e-05, 0.05692074646698261, 11.171973798842838, 183.60608782036712] 0.9111
[8.02981653523445e-05, 0.056591187111412006, 11.20557248878498, 183.60608782036712] 0.9099
[8.02981653523445e-05, 0.056591187111412006, 11.171973798842838, 183.68639221138548] 0.9098
[8.105561948615312e-05, 0.056261627755841405, 11.18877314381391, 183.6462400158763] 0.9104
[8.086625595270096e-05, 0.0564264074336267, 11.184573307571142, 183.636201966999] 0.91
[8.048752888579665e-05, 0.0567559667891973, 11.176173635085606, 183.61612586924443] 0.9103
[8.077157418597488e-05, 0.056508797272519354, 11.182473389449758, 183.63118294256037] 0.9098
[7.901996150154246e-05, 0.05654999219196568, 11.19402293911737, 183.6587875769729] 0.9099
[7.971823953114728e-05, 0.05656029092182726, 11.188510654048736, 183.64561263782147] 0.9099

In [22]:
# Run once at the end to ensure the directory of experiments reflects the solution case
fn2(sol.x)

[8.02981653523445e-05, 0.056591187111412006, 11.171973798842838, 183.60608782036712] 0.9098


0.9098

In [18]:
x = [8.08E-05, 0.056338207, 11.00500268, 183.8237737]
x0 = norm_x0(x, bounds)
fn2(x0)

[8.079999999999999e-05, 0.056338207, 11.00500268, 183.8237737] 0.9107


0.9107